Required packets

In [1]:

from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.stats import yeojohnson

## Prepare the dataset

Load the dataset and retrieve header and time

In [ ]:
#date_parser = lambda x: to_datetime(x, format="%d/%m/%Y")
series = pd.read_csv('drive/MyDrive/dataset/Custom/H_test_noise.csv', header=0, index_col=0, parse_dates=True)

series.replace('.', np.nan, inplace=True)

# Check for NaN values
nan_check = series.isna().any().any()

if nan_check:
    print("Dataset contains NaN values")
    upsampled= series.resample('D').mean()
    series = upsampled.interpolate(method = 'linear')
else:
    print("Dataset does not contain NaN values")

# Retrieve the header (column names) from the original DataFrame
header = series.columns.tolist()
print(header)

# Create a time index for the entire period
time_index = series.index

X = series.values

plt.plot(time_index, X)

plt.title("Univariate Dataset (NaN interpolated)")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.show()

Remove the change in variance and the trend. 

In [ ]:
# To remove change in veriance
xt, lmbda = yeojohnson(X)
# To remove trend
model = LinearRegression()
y = np.array([x for x in range(0, len(xt))])
model.fit(y.reshape(-1, 1), xt)
pred = model.predict(y.reshape(-1, 1))
detrend = xt - pred
# Plot
plt.plot(detrend)
plt.show()

Scale the time series, need to obtain the base configuration

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1)) # Scale the values between -1 and 1
#time_series_scaled = scaler.fit_transform(detrended.values.reshape(-1, 1))
time_series_scaled = scaler.fit_transform(detrend.reshape(-1, 1))

Create the new dataset

In [ ]:
df = DataFrame(data = time_series_scaled[:20000], index = time_index[:20000], columns = header)
df.to_csv('series_normalized.csv')

Plot

In [ ]:
plt.plot(time_index, time_series_scaled)
plt.title('Dataset')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.show()